# SI 618 Final Project Notebook

## Project Description

Our team is interested in understanding hotel booking trends to provide business insights for management and stakeholder groups.

Primary Question:
How can we use the hotel bookings data to improve business operations through prediction of guest cancellation?

Our goal is to inform hotel management, stakeholders, tourism sector the following:
* Data driven customer segmentation / clusters
    * Top Customer Profiles
    * Demographic Information
    * Positioning for Key Performance amongst customers
* Booking cancellation trends amongst hotel guests
    * Cancellation prediction based on historical data
    * Human interpretability around cancellation classification
    * Define potential strategic goals based on operational objectives (e.g., Accuracy, Precision, Sensitivity / Recall, Specificity)


## Python Set Up

#### Import and load libraries

In [222]:
# Load in libraries
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

## Load, Inspect and Clean Data

### Load, Inspect and Clean Objectives
* Transform existing data into bucketed data, if necessary, to add features to the prediction modeling
* Normalizing numerical data using log transformations and or other kinds of numerical transformations
* Handle missing data by either:
    * Removing missing observations
    * Imputing missing observations with the median value (if numerical)
    * Imputing missing observations with the mode or random sample of distribution of categories (if categorical)

#### Load in the data

In [223]:
# Load in the hotel bookings data set

# Change this path to where you have the hotel bookings data set
hotel_bookings = pd.read_csv('C:/VS_Code_Projects/SI_618/si618_final_project/data/hotel_bookings.csv')

#### Inspect the dataset

##### Look at the first few rows of the data

In [224]:
# Look at the first few rows of the data set
hotel_bookings.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


##### Look at the data types of the differnet features in the data set

In [225]:
# Look at the data types of each column
hotel_bookings.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

##### Convert some of the features to the appropriate data types

In [226]:
# convert columns to the appropritat data types given the type of data it is
# more specifically turn  the following columns into categorical data types
# arrival_date_year, arrival_date_week_number, agent, company, arrival_date_day_of_month
hotel_bookings['arrival_date_year'] = hotel_bookings['arrival_date_year'].astype('object')
hotel_bookings['agent'] = hotel_bookings['agent'].astype('object')
hotel_bookings['company'] = hotel_bookings['company'].astype('object')

##### Re-inspect the data types of the differnet features in the data set

In [227]:
# Look at the data types of each column
hotel_bookings.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                  object
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

##### Look at the shape of the data set

In [228]:
# Look at the number of rows and columns in the data set
hotel_bookings.shape

(119390, 32)

##### Look at the number of unique values for each feature of the data set

In [229]:
# check the unique values of each column
hotel_bookings.nunique()

hotel                                2
is_canceled                          2
lead_time                          479
arrival_date_year                    3
arrival_date_month                  12
arrival_date_week_number            53
arrival_date_day_of_month           31
stays_in_weekend_nights             17
stays_in_week_nights                35
adults                              14
children                             5
babies                               5
meal                                 5
country                            177
market_segment                       8
distribution_channel                 5
is_repeated_guest                    2
previous_cancellations              15
previous_bookings_not_canceled      73
reserved_room_type                  10
assigned_room_type                  12
booking_changes                     21
deposit_type                         3
agent                              333
company                            352
days_in_waiting_list     

In [230]:
# look at the actual unique values of each column
for i in hotel_bookings.columns:
    print(i)
    print(hotel_bookings[i].unique())
    print('\n')

hotel
['Resort Hotel' 'City Hotel']


is_canceled
[0 1]


lead_time
[342 737   7  13  14   0   9  85  75  23  35  68  18  37  12  72 127  78
  48  60  77  99 118  95  96  69  45  40  15  36  43  70  16 107  47 113
  90  50  93  76   3   1  10   5  17  51  71  63  62 101   2  81 368 364
 324  79  21 109 102   4  98  92  26  73 115  86  52  29  30  33  32   8
 100  44  80  97  64  39  34  27  82  94 110 111  84  66 104  28 258 112
  65  67  55  88  54 292  83 105 280 394  24 103 366 249  22  91  11 108
 106  31  87  41 304 117  59  53  58 116  42 321  38  56  49 317   6  57
  19  25 315 123  46  89  61 312 299 130  74 298 119  20 286 136 129 124
 327 131 460 140 114 139 122 137 126 120 128 135 150 143 151 132 125 157
 147 138 156 164 346 159 160 161 333 381 149 154 297 163 314 155 323 340
 356 142 328 144 336 248 302 175 344 382 146 170 166 338 167 310 148 165
 172 171 145 121 178 305 173 152 354 347 158 185 349 183 352 177 200 192
 361 207 174 330 134 350 334 283 153 197 133 241 193 235

##### Inspect the number of null values for each feature of the data set

In [231]:
# check for missing values in each column
hotel_bookings.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

##### Handle the missing data

In [232]:
# show me the missing observations for the children column
hotel_bookings = hotel_bookings[~hotel_bookings['children'].isnull()].reset_index(drop=True)

##### Inspect the distributions of each of the numerical and categorical features

In [233]:
# look at the distribution of numerical columns
hotel_bookings.describe()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000,119386.000000
mean,0.370395,104.014801,27.165003,15.798553,0.927605,2.500310,1.856390,0.103890,0.007949,0.031913,0.087121,0.137102,0.221131,2.321227,101.833541,0.062520,0.571340
std,0.482913,106.863286,13.605334,8.780783,0.998618,1.908289,0.579261,0.398561,0.097438,0.175770,0.844350,1.497462,0.652315,17.595011,50.534664,0.245295,0.792798
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,94.590000,0.000000,0.000000
75%,1.000000,160.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,391.000000,5400.000000,8.000000,5.000000


In [234]:
# look at the distribution of categorical columns
hotel_bookings.describe(include='object')

,hotel,arrival_date_year,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,agent,company,customer_type,reservation_status,reservation_status_date
count,119386,119386,119386,119386,118898,119386,119386,119386,119386,119386,103048.0,6797.0,119386,119386,119386
unique,2,3,12,5,177,7,5,10,12,3,333.0,352.0,4,3,926
top,City Hotel,2016,August,BB,PRT,Online TA,TA/TO,A,A,No Deposit,9.0,40.0,Transient,Check-Out,2015-10-21
freq,79326,56707,13873,92306,48586,56476,97870,85994,74053,104637,31960.0,927.0,89613,75166,1461


##### Plot the numerical features to visually inspect the distributions

In [235]:
# use subplots to plot the distribution of each of the numerical columns in a histogram
# this will allow us to see the distribution of each column in one plot
# fig, ax = plt.subplots(3, 4, figsize=(30, 20))

# create a list of the numerical columns
# num_cols = hotel_bookings.select_dtypes(include='number').columns

# create a counter to keep track of the row and column we are on
# row = 0
# col = 0

# loop through each of the numerical columns
# for i in num_cols:
    # plot the histogram of the column
#     ax[row, col].hist(hotel_bookings[i], bins=20)
    # set the title of the plot to the name of the column
#     ax[row, col].set_title(i)
    # increase the column counter by 1
#     col += 1
    # if the column counter is equal to 3
#     if col == 4:
        # reset the column counter to 0
#         col = 0
        # increase the row counter by 1
#         row += 1

# show the plots
# plt.show()

##### Drop the features that are not needed for the analysis

In [236]:
# drop columns in dataset
# drop the following columns because they are not useful for our analysis
# agent, company arrival_date_year reservation_status_date reservation_status

# drop the agent column
hotel_bookings.drop('agent', axis=1, inplace=True)

# drop the company column
hotel_bookings.drop('company', axis=1, inplace=True)

# drop the arrival_date_year column
hotel_bookings.drop('arrival_date_year', axis=1, inplace=True)

# drop the reservation_status_date column
hotel_bookings.drop('reservation_status_date', axis=1, inplace=True)

# drop the reservation_status column
hotel_bookings.drop('reservation_status', axis=1, inplace=True)

In [237]:
hotel_bookings.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
days_in_waiting_list                int64
customer_type                     

##### Convert the categorical features to dummy variables

In [238]:
hotel_type = pd.get_dummies(hotel_bookings['hotel'], drop_first=True, dtype=int)

hotel_arrival_date_month = pd.get_dummies(hotel_bookings['arrival_date_month'], drop_first=True, dtype=int)

hotel_meal = pd.get_dummies(hotel_bookings['meal'], drop_first=True, dtype=int)

hotel_country = pd.get_dummies(hotel_bookings['country'], drop_first=True, dtype=int)

hotel_market_segment = pd.get_dummies(hotel_bookings['market_segment'], drop_first=True, dtype=int)

hotel_distribution_channel = pd.get_dummies(hotel_bookings['distribution_channel'], drop_first=True, dtype=int)

hotel_reserved_room_type = pd.get_dummies(hotel_bookings['reserved_room_type'], drop_first=True, dtype=int)

hotel_assigned_room_type = pd.get_dummies(hotel_bookings['assigned_room_type'], drop_first=True, dtype=int)

hotel_deposit_type = pd.get_dummies(hotel_bookings['deposit_type'], drop_first=True, dtype=int)

hotel_customer_type = pd.get_dummies(hotel_bookings['customer_type'], drop_first=True, dtype=int)

##### Column bind the dummary variables to the original data set

In [239]:
# column bind dummy variables to original data set
hotel_bookings = pd.concat([
    hotel_bookings,
    hotel_type,
    hotel_arrival_date_month,
    hotel_meal,
    hotel_country,
    hotel_market_segment,
    hotel_distribution_channel,
    hotel_reserved_room_type,
    hotel_assigned_room_type,
    hotel_deposit_type,
    hotel_customer_type
], axis=1)

##### Drop the original categorical features from the data set that have been converted to dummy variables

In [240]:
# drop original columns that have been converted to dummy variables from the hotel_books data set
hotel_bookings.drop([
    'hotel',
    'arrival_date_month',
    'meal',
    'country',
    'market_segment',
    'distribution_channel',
    'reserved_room_type',
    'assigned_room_type',
    'deposit_type',
    'customer_type'
], axis=1, inplace=True)


In [248]:
# show me the missing values in each column that are greater than 0
hotel_bookings.isnull().sum()[hotel_bookings.isnull().sum() > 0]

Series([], dtype: int64)

In [ ]:
hotel_bookings

## Test and Train Split

#### Set Seed

In [241]:
# Set random seed
random.seed(23)

#### Shuffle the data

In [242]:
# shuffle the data to insure that the data is not ordered in any way
shuffle = random.sample(list(hotel_bookings.index), int(hotel_bookings.shape[0]))

hotel_bookings_shuffled = hotel_bookings.iloc[shuffle, :]

shuffle = random.sample(list(hotel_bookings.index), int(hotel_bookings.shape[0]))

hotel_bookings_shuffled = hotel_bookings.iloc[shuffle, :]

#### Split the data into test and train

In [ ]:
# set the size of the data subset
train_size = 0.8

# Subset the data set
hotel_bookings.shape[0]

# randomly sample 80% of the row indices from the hotel bookings data set
# this will be used to subset the data set
train_obs = random.sample(list(hotel_bookings.index), int(hotel_bookings.shape[0] * train_size))

# Create the training set
training_set = hotel_bookings.iloc[train_obs, :]

# Create the testing set
testing_set = hotel_bookings.iloc[~hotel_bookings.index.isin(train_obs), :]

#### Look at the first few rows of the train and test data

In [243]:
training_set.head()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,H,I,K,L,P,Non Refund,Refundable,Group,Transient,Transient-Party
70945,1,260,26,26,1,1,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
12268,1,18,25,21,1,4,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
33210,0,207,7,13,1,3,1,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
102632,0,73,49,3,0,1,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
3343,1,149,49,5,2,1,2,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [244]:
testing_set.head()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,H,I,K,L,P,Non Refund,Refundable,Group,Transient,Transient-Party
2,0,7,27,1,0,1,1,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,14,27,1,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,9,27,1,0,2,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,1,75,27,1,0,3,2,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
13,0,18,27,1,0,4,2,1.0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Build Classification Models

#### Logistic Regression

##### Preprocess data X_log and y_log

In [245]:
# Create Predictor variables numpy Array training_set_X_log
training_set_X_log = training_set.drop(['is_canceled'], axis=1)

training_set_y_log = training_set['is_canceled']


##### Fit the model

In [246]:
model_log_train = LogisticRegression()
model_log_train.fit(training_set_X_log, training_set_y_log)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values